This notebook is used to display an image in the dataset

In [1]:
import json
import os
import pandas as pd
import ipyplot
from tools import url_to_image_id_map_helper
from IPython.display import Image, HTML, display

First, set the `expr` to the experiment name:
- dog3_non_random_with_init
- dog3_non_random_from_scratch
- dog3_random_with_init
- birds

In [2]:
expr = "birds"

In [3]:
with open(os.path.join("results", expr + "_" + "tracker.json")) as tracker_handle:
    tracker = json.load(tracker_handle)

In [4]:
with open(os.path.join("results", expr + "_" + "worker.json")) as worker_handle:
    workers = json.load(worker_handle)

In [5]:
url_to_image_id_map = url_to_image_id_map_helper()

In [6]:
url_to_predicted_cluster_id_map = {}
for predicted_cluster_id, predicted_cluster in enumerate(tracker["clusters"]):
    for image in predicted_cluster:
        image_url = image["value"]
        url_to_predicted_cluster_id_map[image_url] = predicted_cluster_id
for hard_question in tracker["hard"]:
    hard_url = hard_question.split(",")[0]
    url_to_predicted_cluster_id_map[hard_url] = -1
    url_to_predicted_cluster_id_map["." + hard_url] = -1
    url_to_predicted_cluster_id_map[".." + hard_url] = -1

## Display the image

Set `image_url` to the url of the image that you want to look at. You can first use the notebook `images_in_predicted_cluster` or `hard_images` to access to a set of urls. Note that the images url for the birds should not include the . at the beginning, i.e. image url should look like `/Birds/44.jpg` instead of `./Birds/44.jpg`, if working on birds20 images.

In [7]:
image_url = "/Birds/44.jpg"

In [8]:
ipyplot.plot_images([("." if "Birds" in image_url else "") + image_url],
                    [str(url_to_image_id_map[image_url]) + ", predicted cluster: %d" % url_to_predicted_cluster_id_map[image_url]],
                    max_images=1,
                    img_width=300)

## Display the images that this image has been queried with

In [9]:
num_clusters = len(tracker["clusters"])

In [10]:
queried_with = {i: [] for i in range(num_clusters)}

In [11]:
for worker in workers:
    for answer in worker["result"]:
        left_url = answer["pair"]["left"]
        right_url = answer["pair"]["right"]
        classification = answer["classification"]
        if left_url == image_url:
            queried_with[url_to_predicted_cluster_id_map[right_url]].append((right_url, classification))

In [12]:
print("This image has been queried with images from the following clusters:")
list(map(lambda x: x[0], (filter(lambda x: len(x[1]) != 0, queried_with.items()))))

This image has been queried with images from the following clusters:


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

Set `queried_with_cluster` to one of the cluster id in the list printed above.

In [13]:
queried_with_cluster = 5

In [14]:
print("Queried with Cluster ID: %d" % queried_with_cluster)
for url, classification in queried_with[queried_with_cluster]:
    if "Birds" in url:
        url = "." + url
    display(HTML(f"""
        <div class="row">
            <h5>Worker's Answer: {classification}</h5>
            <img src={url} style="width:250px"/>
        </div>
    """))

Queried with Cluster ID: 5
